In [1]:
reset = false

PROJECT_DIR = abspath(joinpath("..", "envs", "RL"))
reset && isdir(PROJECT_DIR) && rm(PROJECT_DIR; recursive=true, force=true)

PROJECT_DIR

"/run/media/cavani/cavani_hdx/julia-abc/envs/RL"

In [2]:
using Pkg
Pkg.activate(PROJECT_DIR)

 Activating environment at `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Project.toml`


In [3]:
deps = [
    "ReinforcementLearning",
    "Flux",
    "CuArrays",
    "CUDAnative",
    "CUDAdrv",
    "Plots",
    "PyPlot",
]

Pkg.add(deps)

   Updating registry at `~/Workspace/julia-abc/software/julia-env/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Project.toml`
 [no changes]
   Updating `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Manifest.toml`
 [no changes]


In [4]:
Pkg.status()

Status `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Project.toml`
  [c5f51814] CUDAdrv v6.3.0
  [be33ccc6] CUDAnative v3.1.0
  [3a865a2d] CuArrays v2.2.0
  [587475ba] Flux v0.10.4
  [91a5bcdd] Plots v1.3.2
  [d330b81b] PyPlot v2.9.0
  [158674fc] ReinforcementLearning v0.4.0


In [5]:
ENV["PYTHON"] = joinpath(ENV["CONDA_INSTDIR"], "envs", "python", "bin", "python")
Pkg.update()
Pkg.API.precompile()
Pkg.status()

   Updating registry at `~/Workspace/julia-abc/software/julia-env/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Updating `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Project.toml`
 [no changes]
   Updating `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Manifest.toml`
 [no changes]
Precompiling project...


Status `/run/media/cavani/cavani_hdx/julia-abc/envs/RL/Project.toml`
  [c5f51814] CUDAdrv v6.3.0
  [be33ccc6] CUDAnative v3.1.0
  [3a865a2d] CuArrays v2.2.0
  [587475ba] Flux v0.10.4
  [91a5bcdd] Plots v1.3.2
  [d330b81b] PyPlot v2.9.0
  [158674fc] ReinforcementLearning v0.4.0


In [6]:
JULIA_BIN = joinpath(ENV["JULIA_INSTDIR"], "bin", "julia")
for (_, pkg) in Pkg.dependencies()
    pkg.is_direct_dep && pkg.version !== nothing || continue
    try
        pkg_name = pkg.name
        run(`$JULIA_BIN --project=$PROJECT_DIR -e "println(\"$pkg_name\"); @time import $pkg_name"`)
    catch
    end
end

CUDAnative
  1.913322 seconds (1.38 M allocations: 78.606 MiB)
Plots
 20.728284 seconds (20.08 M allocations: 1.062 GiB, 2.58% gc time)
ReinforcementLearning
 40.826167 seconds (61.64 M allocations: 3.279 GiB, 3.87% gc time)
PyPlot
  6.607068 seconds (6.15 M allocations: 314.018 MiB, 1.46% gc time)
CuArrays
  6.178623 seconds (7.52 M allocations: 430.870 MiB, 2.03% gc time)
CUDAdrv
  0.341961 seconds (185.24 k allocations: 12.019 MiB)
Flux
 33.963103 seconds (55.79 M allocations: 2.991 GiB, 4.58% gc time)


In [7]:
using ReinforcementLearning
using Statistics: mean

env = CartPoleEnv(; T=Float32, seed = 123)

agent = Agent(
    policy = RandomPolicy(env; seed = 456),
    trajectory = CircularCompactSARTSATrajectory(; capacity = 3, state_type = Float32, state_size = (4,)),
)

hook = ComposedHook(TotalRewardPerEpisode(), TimePerStep())

run(agent, env, StopAfterEpisode(10_000), hook)

@info "stats for random policy" avg_reward = mean(hook[1].rewards) avg_fps = 1 / mean(hook[2].times)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
┌ Info: stats for random policy
│   avg_reward = 21.0591
│   avg_fps = 1.1354088607307489e6
└ @ Main In[7]:15
